In [2]:
from nltk.tokenize import word_tokenize
from konlpy.tag import Okt
import nltk

In [3]:
okt = Okt()

In [26]:
def load_data(file_path):
    train = []
    
    count = 0
    
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            if count == 500:
                break
                
            line = line.strip()
            id, doc, label = line.split('\t')
            if label == '1': 
                label = 'pos'
            elif label == '0': 
                label = 'neg'
            train.append((doc, label))
            count += 1
    return train

In [27]:
def pos_tokenize(raw_sent):
    pos_sent = []
    sent = okt.pos(raw_sent, norm=True, stem=True)
    
    for tup in sent:
        word, tag = tup[0], tup[1]
        word_tag = word + '/' + tag
        pos_sent.append(word_tag)
    return ' '.join(pos_sent)

In [28]:
def make_word_dict(train, use_morph=False):
    all_words = set()
    
    for tup in train:
        sent, label = tup[0], tup[1]
        if use_morph:
            sent = pos_tokenize(sent)
        words = word_tokenize(sent)
        for word in words:
            all_words.add(word)
    return all_words

In [29]:
def make_train_feats(train, all_words, use_morph=False):
    train_features = []
    
    for tup in train:
        sent, label = tup[0], tup[1]
        if use_morph:
            sent = pos_tokenize(sent)
        words = word_tokenize(sent)
        tmp = {set_word: (set_word in words) for set_word in all_words}
        sent_tup = (tmp, label)
        train_features.append(sent_tup)
    return train_features

In [32]:
train = load_data('ratings_train.txt')
print(train[:100])

[('document', 'label'), ('아 더빙.. 진짜 짜증나네요 목소리', 'neg'), ('흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', 'pos'), ('너무재밓었다그래서보는것을추천한다', 'neg'), ('교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', 'neg'), ('사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다', 'pos'), ('막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.', 'neg'), ('원작의 긴장감을 제대로 살려내지못했다.', 'neg'), ('별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네', 'neg'), ('액션이 없는데도 재미 있는 몇안되는 영화', 'pos'), ('왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?', 'pos'), ('걍인피니트가짱이다.진짜짱이다♥', 'pos'), ('볼때마다 눈물나서 죽겠다90년대의 향수자극!!허진호는 감성절제멜로의 달인이다~', 'pos'), ('울면서 손들고 횡단보도 건널때 뛰쳐나올뻔 이범수 연기 드럽게못해', 'neg'), ('담백하고 깔끔해서 좋다. 신문기사로만 보다 보면 자꾸 잊어버린다. 그들도 사람이었다는 것을.', 'pos'), ('취향은 존중한다지만 진짜 내생에 극장에서 본 영화중 가장 노잼 노감동임 스토리도 어거지고 감동도 어거지', 'neg'), ('ㄱ냥 매번 긴장되고 재밋음ㅠㅠ', 'pos'), ('참 사람들 웃긴게 바스코가 이기면 락스코라고 까고바비가 이기면 아이돌이라고 깐다.그냥 까고싶어서 안달난것처럼 보인다', 'pos'), ('굿바이 레닌 표절인것은 이해하는데 왜 뒤로 갈수록 재미없어지냐', 'neg'), ('이건 정말 깨알 캐스팅과 질퍽하지않은 산뜻한 내용구성이 잘 버무러진 깨알일드!!♥'

In [9]:
use_morph = True
all_words = make_word_dict(train, use_morph)
train_feature = make_train_feats(train, all_words, use_morph)

In [10]:
import urllib

In [11]:
test = urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt",
    filename="ratings_test.txt"
)

In [12]:
classifier = nltk.NaiveBayesClassifier.train(train_feature)
classifier.show_most_informative_features()

Most Informative Features
                       ; = True              neg : pos    =      8.2 : 1.0
          재미없다/Adjective = True              neg : pos    =      8.2 : 1.0
                주인공/Noun = True              neg : pos    =      7.5 : 1.0
                 최고/Noun = True              pos : neg    =      6.9 : 1.0
                  뭐/Noun = True              neg : pos    =      6.8 : 1.0
           재밌다/Adjective = True              pos : neg    =      6.5 : 1.0
                 내용/Noun = True              neg : pos    =      6.2 : 1.0
       ㅡㅡ/KoreanParticle = True              neg : pos    =      6.1 : 1.0
                스토리/Noun = True              neg : pos    =      6.1 : 1.0
                 다시/Noun = True              pos : neg    =      5.9 : 1.0


In [31]:
# test_sent = '재미없다'
# print(test_sent)
# test_sent = pos_tokenize(test_sent)
# print(test_sent)
# words = word_tokenize(test_sent)
# test_feature = {set_word: (set_word in words) for set_word in all_words}

sents = load_data('ratings_test.txt')
for i in range(3, 100):
    test_sent = sents[i][0]
    test_sent = pos_tokenize(test_sent)
    words = word_tokenize(test_sent)
    test_feature = {set_word: (set_word in words) for set_word in all_words}
    print(classifier.classify(test_feature))

neg
neg
neg
pos
pos
neg
neg
pos
pos
pos
pos
pos
pos
neg
pos
neg
pos
neg
neg
pos
pos
pos
neg
neg
pos
pos
neg
pos
pos
pos
neg
neg
neg
neg
neg
pos
neg
pos
neg
neg
neg
pos
neg
pos
pos
neg
pos
pos
neg
pos
neg
neg
pos
pos
neg
neg
pos
pos
pos
neg
pos
pos
pos
neg
pos
pos
pos
pos
neg
pos
neg
neg
pos
pos
pos
pos
neg
neg
neg
pos
pos
pos
pos
pos
pos
pos
pos
neg
neg
neg
pos
pos
neg
neg
neg
pos
neg


In [18]:

classifier.classify(test_feature)

'neg'